In [8]:
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np

In [9]:
df = pd.read_csv('files/all.csv', index_col='Unnamed: 0')

/Users/inejj/anaconda3/envs/appartementen/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3072: DtypeWarning: Columns (29,37,58,92,116,120,136,139,140,141,144,146,150,152,154,162,169,170,172,173,174,175,177,180,181,182,183,184,186,187,188,189,190,191,193,194,195,196,197,198,200,201,203,205,206,207,209,210,211,212,213) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [10]:

df = df.drop(columns=['flags.main','customerLogoUrl','customerName','media.pictures',
                      'flags.secondary','Price','property.title','publication.size',
                      'publication.visualisationOption','External reference',
                      'Reference number of the EPC report','Website', 'Available as of',
                      'Address'], axis =1)

In [11]:
# Removing outliers
df = df[df['Floor :'] < 30]

# Geen openbare verkopen waar we moeten bieden of verkopen op lijfrente.
df = df[df['price.label'] == 'Price']

# Geen slaapkamers vermeld is nogal raar.

df = df[df['property.bedroomCount'] > 0]

binary_cols = ['Double glazing','Armored door','Elevator','Basement',
             'Intercom','Planning permission obtained', 'Type of building']
df[binary_cols] =df[binary_cols].replace('Yes', 1).replace('No',0)


# Striping things like 'm²square meters' from numerical columns and
# converting them to float

numerical_cols = ['CO² emission', 'Living area','Living room surface',
            'Bedroom 1 surface','Bedroom 2 surface','Cadastral income',
            'E-level (primary energy consumption)','Terrace surface']

strippers = ['kg CO2/m²','kWh/m²kilowatt hour per square meters',
            'm²square meters','€']

for x in numerical_cols:
    for y in strippers:
        df[x] = df[x].str.strip(y).replace('Not specified',np.nan)
    df[x] = df[x].str.replace('.','').str.replace(',','.').replace('', np.nan)
    df[x] = df[x].astype(float)

# Dropping columns with duplicate information
to_drop = ['Floor','price.shortDisplayPrice','price.accessibilityPrice',
           'price.mainDisplayPrice', 'property.location.floor','Bedrooms','bedrooms','bedroom',
           'transaction.sale.price','bathroom','m²square meterslivable space']

df = df.drop(to_drop, axis=1)

In [12]:
for column in df.columns:
    if df[column].isnull().sum() > 0.8*len(df):
        df = df.drop(column, axis=1)
    elif len(df[column].unique()) == 1:
        df = df.drop(column, axis=1)

In [13]:
df

,id,price.mainValue,property.bedroomCount,property.location.box,property.location.latitude,property.location.locality,property.location.longitude,property.location.number,property.location.postalCode,property.location.street,...,As built plan,Bedroom 1 surface,Bedroom 2 surface,Covered parking spaces,Intercom,Terrace surface,Basement,Floor :,Number of floors,Elevator
1,8745713.0,185000.0,2.0,b61,51.185615,Berchem,4.441640,118,2600,Fruithoflaan,...,No,NaN,NaN,1.0,NaN,10.0,1.0,6.0,NaN,1.0
2,8745715.0,280000.0,3.0,b61,51.185615,Berchem,4.441640,118,2600,Fruithoflaan,...,No,NaN,NaN,1.0,NaN,35.0,1.0,6.0,NaN,1.0
4,8744282.0,184000.0,1.0,1R,51.113038,Niel,4.327348,64,2845,Edward Claessenslaan,...,No,NaN,NaN,1.0,NaN,NaN,NaN,1.0,2.0,NaN
5,8664757.0,149000.0,1.0,NaN,51.221663,Antwerp,4.398299,"1, 217",2000,Kaasstraat,...,No,NaN,NaN,NaN,NaN,NaN,NaN,2.0,NaN,1.0
9,8614460.0,175000.0,2.0,NaN,51.305726,Oostmalle,4.737027,74,2390,Hoogstraatsebaan,...,NaN,NaN,NaN,NaN,NaN,5.0,NaN,1.0,NaN,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4158,8289434.0,1644000.0,10.0,NaN,NaN,Antwerp,NaN,NaN,2000,NaN,...,No,NaN,NaN,NaN,NaN,56.0,NaN,0.0,NaN,1.0
4160,7861828.0,645000.0,2.0,NaN,NaN,Antwerp,NaN,NaN,2000,NaN,...,NaN,NaN,NaN,1.0,NaN,47.0,NaN,0.0,NaN,1.0
4163,8710769.0,396500.0,2.0,NaN,NaN,Antwerp,NaN,NaN,2000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,NaN,NaN
4174,8139678.0,1275000.0,16.0,NaN,NaN,Antwerp,NaN,NaN,2000,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,0.0,3.0,NaN


In [14]:
df.to_pickle('files/cleaned.pkl')
df.to_csv('files/cleaned.csv')